# EfficientNet+Augmentation for Cassava Disease Classification using TF.Keras

This notebook presents a full pipeline to load the data, apply advanced data augmentation, train an EfficientNet and use the model to predict over the test images. To make it possible to run within the allocated time for notebooks, this notebook will only present a single fold with a split of 80% for training and 20% for validation. Due to the original image size of 600x800 pixels, we will randomly crop 512x512 images from original images in order to keep the highest image resolution possible for our model training. Previous versions of this notebook used resized images and the results were extremely poor in comparison (~0.42 accuracy).

Below are some version notes which were written at version 20 so I only included what I could remember.

Version notes:
* 27: Fix bug in the TTA process.
* 26: Move from using `ImageDataGenerator` to `tf.data` to load data into the model. This allows to tune the Normalization layer in the pretrained EfficientNetB3 model provided by `tf.keras`. By default, images are normalized using the `imagenet` dataset's mean and standard deviation. *- score: 0.888*
* 24: Test CosineDecay instead of ReduceLRonPlateau and increase the dropout rate within the EfficientNetB3 model. *- score: 0.885*
* 22: Replace the custom generator and data augmentation using `imgaug` with the new Keras preprocessing layers. Also increase the image size from 300x300 to 512x512. *- score: 0.880*
* 21: *- score: 0.883*
* 20: Fix bug activating the "training mode" by default in the custom generator, even during validation. Removed data standardization and class weights, simplify the bottleneck layers of the model, and remove the dropout from the data augmentation techniques. *- score: 0.873*
* 17: Additional data augmentation techniques. *- score: 0.821*
* 16: Tighter scan of the images at test time. *- score: 0.857*
* 15: Add data standardization. (Cannot remember if there was a bug but the score was abnormaly low) *- score: 0.692*
* 11: Train the model by randomly cropping 300x300pixel tiles from the original images *- score: 0.847*
* 9: Classification from resized images. *- score: 0.421* 

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.preprocessing import minmax_scale
import random
import cv2
from imgaug import augmenters as iaa
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop,CenterCrop, RandomRotation

# Prepare the training and validation data generators

In [ ]:
training_folder = '../input/cassava-leaf-disease-classification/train_images/'

In [ ]:
samples_df = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
samples_df = shuffle(samples_df, random_state=42)
samples_df["filepath"] = training_folder+samples_df["image_id"]
samples_df.head()

I keep 80% of the data provided for training and retain the other 20% for validation during my training process.

In [ ]:
training_percentage = 0.8
training_item_count = int(len(samples_df)*training_percentage)
validation_item_count = len(samples_df)-int(len(samples_df)*training_percentage)
training_df = samples_df[:training_item_count]
validation_df = samples_df[training_item_count:]

Initially, I had set the image size to 300x300 in order to fit the original input size to the EfficientnetB3 model. However, as we are randomly cropping these images from the original 600x800-pixel images, it appears that using a dimension of 512x512 pixels leads to better results.

In [ ]:
batch_size = 8
image_size = 512
input_shape = (image_size, image_size, 3)
dropout_rate = 0.4
classes_to_predict = sorted(training_df.label.unique())

The code below allows to load the data from the dataframes using `tf.data`.

In [ ]:
training_data = tf.data.Dataset.from_tensor_slices((training_df.filepath.values, training_df.label.values))
validation_data = tf.data.Dataset.from_tensor_slices((validation_df.filepath.values, validation_df.label.values))

In [ ]:
def load_image_and_label_from_path(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

training_data = training_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
validation_data = validation_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)

In [ ]:
training_data_batches = training_data.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=AUTOTUNE)
validation_data_batches = validation_data.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=AUTOTUNE)

I also prepare a special dataset that will be fed to the Normalization layer. The EfficientnetB3 provided by `tf.keras` includes an out-of-the-box Normalization layer fit onto the `imagenet` dataset. Therefore, we can pull that layer and use the `adapt` function to refit it to the Cassava Disease dataset.

In [ ]:
adapt_data = tf.data.Dataset.from_tensor_slices(training_df.filepath.values)
def adapt_mode(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = layers.experimental.preprocessing.Rescaling(1.0 / 255)(img)
    return img

adapt_data = adapt_data.map(adapt_mode, num_parallel_calls=AUTOTUNE)
adapt_data_batches = adapt_data.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=AUTOTUNE)

The data augmentation preprocessing layers below will be used when training the model but disabled in inference mode.

In [ ]:
data_augmentation_layers = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomCrop(height=image_size, width=image_size),
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomRotation(0.25),
        layers.experimental.preprocessing.RandomZoom((-0.2, 0)),
        layers.experimental.preprocessing.RandomContrast((0.2,0.2))
    ]
)

Simply reusing some of the code from [this tutorial](https://www.tensorflow.org/tutorials/images/data_augmentation) to show what our augmentations look like. I add the image previously opened to a batch and pass it through the data augmentation layers.

In [ ]:
image = Image.open("../input/cassava-leaf-disease-classification/train_images/3412658650.jpg")
plt.imshow(image)
plt.show()

In [ ]:
image = tf.expand_dims(np.array(image), 0)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation_layers(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0])
  plt.axis("off")

# Build the model

I am using an EfficientNetB3 on top of which I add some outputs layers to predict our 5 disease classes. I decided to load the imagenet pretrained weights locally to keep the internet off (part of the requirements to submit a kernel to this competition).

At this point, you may have noticed that I have not used any kind of normalization or rescaling. I recently discovered that there is a Normalization layer included in Keras'pretrained EfficientNet, as mentioned [here](https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/#keras-implementation-of-efficientnet).

In [ ]:
efficientnet = EfficientNetB3(weights="../input/efficientnetb3-notop/efficientnetb3_notop.h5", 
                              include_top=False, 
                              input_shape=input_shape, 
                              drop_connect_rate=dropout_rate)

inputs = Input(shape=input_shape)
augmented = data_augmentation_layers(inputs)
efficientnet = efficientnet(augmented)
pooling = layers.GlobalAveragePooling2D()(efficientnet)
dropout = layers.Dropout(dropout_rate)(pooling)
outputs = Dense(len(classes_to_predict), activation="softmax")(dropout)
model = Model(inputs=inputs, outputs=outputs)
    
model.summary()

The 3rd layer of the Efficientnet is the Normalization layer, which can be tuned to our new dataset instead of `imagenet`. Be patient on this one, it does take a bit of time as we're going through the entire training set.

In [ ]:
%%time
model.get_layer('efficientnetb3').get_layer('normalization').adapt(adapt_data_batches)

I wanted to try the new `CosineDecay` function implemented in `tf.keras` as it seemed promising and I struggled to find the right settings (if there were any) for the `ReduceLROnPlateau`.

In [ ]:
epochs = 8
decay_steps = int(round(len(training_df)/batch_size))*epochs
cosine_decay = CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.3)

callbacks = [ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(cosine_decay), metrics=["accuracy"])

In [ ]:
history = model.fit(training_data_batches,
                  epochs = epochs, 
                  validation_data=validation_data_batches,
                  callbacks=callbacks)

# Verification of the training process

First, we will check that we perform on similar level on both the training and validation. The training curve will also tell us if we stopped training too early or may have overfitted in comparison to the validation data.

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss over epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.show()

We load the best weights that were kept from the training phase. Just to check how our model is performing, we will attempt predictions over the validation set. This can help to highlight any classes that will be consistently miscategorised.

In [ ]:
model.load_weights("best_model.h5")

# Prediction on test images

In [ ]:
def scan_over_image(img_path, crop_size=512):
    '''
    Will extract 512x512 images covering the whole original image
    with some overlap between images
    '''
    
    img = Image.open(img_path)
    img_height, img_width = img.size
    img = np.array(img)
    
    y = random.randint(0,img_height-crop_size)
    x = random.randint(0,img_width-crop_size)

    x_img_origins = [0,img_width-crop_size]
    y_img_origins = [0,img_height-crop_size]
    img_list = []
    for x in x_img_origins:
        for y in y_img_origins:
            img_list.append(img[x:x+crop_size , y:y+crop_size,:])
  
    return np.array(img_list)

In [ ]:
def display_samples(img_path):
    '''
    Display all 512x512 images extracted from original images
    '''
    
    img_list = scan_over_image(img_path)
    sample_number = len(img_list)
    fig = plt.figure(figsize = (8,sample_number))
    for i in range(0,sample_number):
        ax = fig.add_subplot(2, 4, i+1)
        ax.imshow(img_list[i])
        ax.set_title(str(i))
    plt.tight_layout()
    plt.show()

display_samples("../input/cassava-leaf-disease-classification/train_images/3412658650.jpg")

I apply some very basic test time augmentation to every local image extracted from the original 600x800 image. We know we can do some fancy augmentation with `imgaug` or `albumentations` but I wanted to do that exclusively with Keras' preprocessing layers to keep the "cleanest" pipeline possible.

In [ ]:
test_time_augmentation_layers = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomZoom((-0.2, 0)),
        layers.experimental.preprocessing.RandomContrast((0.2,0.2))
    ]
)

In [ ]:
def predict_and_vote(image_filename, folder, TTA_runs=4):
    '''
    Run the model over 4 local areas of the given image,
    before making a decision depending on the most predicted
    disease.
    '''
    
    #apply TTA to each of the 4 images and sum all predictions for each local image
    localised_predictions = []
    local_image_list = scan_over_image(folder+image_filename)
    for local_image in local_image_list:
        duplicated_local_image = tf.convert_to_tensor(np.array([local_image for i in range(TTA_runs)]))
        augmented_images = test_time_augmentation_layers(duplicated_local_image)
        
        predictions = model.predict(augmented_images)
        localised_predictions.append(np.sum(predictions, axis=0))
    
    #sum all predictions from all 4 images and retrieve the index of the highest value
    global_predictions = np.sum(np.array(localised_predictions),axis=0)
    final_prediction = np.argmax(global_predictions)
    
    return final_prediction

In [ ]:
def run_predictions_over_image_list(image_list, folder):
    predictions = []
    with tqdm(total=len(image_list)) as pbar:
        for image_filename in image_list:
            pbar.update(1)
            predictions.append(predict_and_vote(image_filename, folder))
    return predictions

First, I test my entire prediction pipeline on the validation set as we have little visibility over the test set.

In [ ]:
validation_df["results"] = run_predictions_over_image_list(validation_df["image_id"], training_folder)

In [ ]:
!cat ../input/cassava-leaf-disease-classification/label_num_to_disease_map.json

In [ ]:
validation_df[:20]

In [ ]:
true_positives = 0
prediction_distribution_per_class = {"0":{"0": 0, "1": 0, "2":0, "3":0, "4":0},
                                     "1":{"0": 0, "1": 0, "2":0, "3":0, "4":0},
                                     "2":{"0": 0, "1": 0, "2":0, "3":0, "4":0},
                                     "3":{"0": 0, "1": 0, "2":0, "3":0, "4":0},
                                     "4":{"0": 0, "1": 0, "2":0, "3":0, "4":0}}
number_of_images = len(validation_df)
for idx, pred in validation_df.iterrows():
    if int(pred["label"]) == pred.results:
        true_positives+=1
    prediction_distribution_per_class[str(pred["label"])][str(pred.results)] += 1
print("accuracy: {}%".format(true_positives/number_of_images*100))

In [ ]:
prediction_distribution_per_class

We can also have a better understanding of where this new model misclassifies diseases by plotting a heatmap from the results. Each row in this heatmap is normalised to highlight the classification distribution per disease without being bothered by the fact that the dataset is imbalanced.

In [ ]:
heatmap_df = pd.DataFrame(columns={"groundtruth","prediction","value"})
for key in prediction_distribution_per_class.keys():
    for pred_key in prediction_distribution_per_class[key].keys():
        value = prediction_distribution_per_class[key][pred_key]/validation_df.query("label==@key").count()[0]
        heatmap_df = heatmap_df.append({"groundtruth":key,"prediction":pred_key,"value":value}, ignore_index=True)   

heatmap = heatmap_df.pivot(index='groundtruth', columns='prediction', values='value')
sns.heatmap(heatmap,cmap="Blues")

In [ ]:
test_folder = '../input/cassava-leaf-disease-classification/test_images/'
submission_df = pd.DataFrame(columns={"image_id","label"})
submission_df["image_id"] =  os.listdir(test_folder)
submission_df["label"] = 0

In [ ]:
submission_df["label"] = run_predictions_over_image_list(submission_df["image_id"], test_folder)

In [ ]:
submission_df

In [ ]:
submission_df.to_csv("submission.csv", index=False)

### Thanks for reading this notebook! If you found this notebook helpful, please give it an upvote. It is always greatly appreciated!